In [ ]:
"""
📘 1. Core Understanding (Essential for LangChain Tools & Agents)
│
├── What is Pydantic?
├── Role in LangChain/LangGraph:
│   ├── Tool Input Schema
│   ├── Output Parsers
│   ├── Agent State validation
│   ├── Prompt-to-Tool interaction structure
└── Type-safety and validation in chains

📦 2. BaseModel (Backbone of LangChain Tool Schemas)
│
├── Defining Input/Output schemas for:
│   ├── @tool decorators
│   ├── Runnable interfaces
│   ├── LangGraph states and memory
├── BaseModel class usage
├── model_dump()
├── model_validate()
├── model_copy()
└── model_config (ConfigDict with validation modes)

📄 3. Model Fields (Tool Parameters & Agent State)
│
├── Type-annotated fields (str, int, float, list, etc.)
├── Optional/Required fields (Optional[])
├── Default values / default_factory
├── Field alias (for prompt or tool parameter naming)
├── Field metadata:
│   ├── description → Used in tool docs/UI
│   ├── examples → Used in LangGraph/LLM preview
└── Field() for constraints (e.g., min_length, ge)

🎯 4. Validation (Ensures tool/agent schema consistency)
│
├── Validation lifecycle in agent execution
├── typing.Annotated for validation hints
├── Built-in constraints: ge, le, min_length, etc.
├── @field_validator (before/after LLM call)
├── @model_validator (cross-field checks)
└── Nested model validation (multi-input tools)

🧬 5. Model Composition (Multi-step Chains & Memory)
│
├── Nested models for structured inputs/outputs
├── Recursive models for nested tool calls or state trees
├── Generic models (not common but usable)
└── Inheritance for reusing base schemas (e.g., BaseToolSchema)

📤 6. Serialization & Deserialization (Crucial for LangGraph state)
│
├── model_dump() → serialize state or tool output
├── model_dump_json() → serialize for storage
├── model_validate() → validate incoming tool inputs
├── @field_serializer (for formatting output)
└── Serialization aliases (for prompt or UI formatting)

🛡️ 7. Strict Mode & Type Safety (Prevents hallucinations or misuse)
│
├── StrictStr, StrictInt for rigid input parsing
├── Prevent coercion from LLM float→str, etc.
├── Useful for LangChain+LangGraph guardrails
└── pydantic-core: Faster validation for runtime agents

🔗 8. Advanced Usage (LangGraph-Specific)
│
├── Discriminated Unions (agent state switching)
│   └── Useful in multi-agent orchestration
├── Private attributes (`PrivateAttr`)
│   └── Store internal agent metadata (e.g., step count)
├── Computed fields (`@computed_field`)
│   └── On-the-fly state fields (e.g., elapsed time, token usage)
└── __pydantic_fields__ → Introspection during tool chaining

📚 9. Error Handling (For Agent Reliability)
│
├── ValidationError handling in tools and LangGraph
├── Custom error messages for LLM feedback
└── LangChain tool fallback based on validation failure

🔌 10. LangChain & LangGraph Integration Points (Direct Usage)
│
├── LangChain:
│   ├── @tool(..., args_schema=MySchema)
│   ├── OutputParser using BaseModel
│   └── Prompt Templates with schema injection
│
├── LangGraph:
│   ├── Agent state classes → subclass BaseModel
│   ├── State transitions → validated input/output
│   ├── Memory schema → pydantic-validated slots
│   └── Persistent state handling with serialization
└── Runnable interfaces → schema-driven pipelines

🧪 11. Testing & Debugging in Agentic AI
│
├── Validate simulated tool inputs
├── Unit testing for BaseModel tool schemas
├── model_dump() for internal state inspection
└── Test cases for multi-agent interactions

🎓 12. Real-World Agentic AI Examples
│
├── Tool input validation (e.g., search query schema)
├── Multi-input schema for LangGraph planner
├── Dynamic function calling validation
├── Structured memory using BaseModel
└── LangChain agent tool I/O with constraints

"""


---

# 📘 **1. Core Understanding of Pydantic**

---

### ✅ 1. Definition

**Pydantic v2** is a Python library that provides runtime **data validation and parsing using type hints**. It's built on a fast core (`pydantic-core`) and is widely used for defining structured data models with strict validation.

---

### 🧰 2. Built-in Functions (Essentials Only)

| Function                | Description                                                       |
| ----------------------- | ----------------------------------------------------------------- |
| `BaseModel`             | Base class to define schemas                                      |
| `.model_validate(data)` | Validates and parses data into a model instance                   |
| `.model_dump()`         | Serializes model data into a dictionary                           |
| `.model_dump_json()`    | Serializes model into JSON                                        |
| `.model_copy()`         | Copies the model (optionally with updates)                        |
| `ConfigDict`            | Used to configure model behavior like `frozen`, `extra`, `strict` |

---

### 🤖 3. Use in Generative AI & Agentic AI (LangChain / LangGraph)

| Where it's used                | Description                                                    |
| ------------------------------ | -------------------------------------------------------------- |
| ✅ Tool Input Schemas           | Define tool input structure using `@tool(args_schema=MyModel)` |
| ✅ Output Parsing               | Use BaseModel to parse structured outputs from LLMs            |
| ✅ Agent State Models           | LangGraph state is represented as BaseModel subclass           |
| ✅ Memory Tracking              | Structured memory slots validated using BaseModel              |
| ✅ Dynamic LLM Function Calling | Enables strict validation of function arguments passed by LLMs |

---

### 🌐 4. Real-Time Use Case

A **LangChain agent** powered by GPT-4 needs to use a weather API tool. The tool expects inputs like `city: str` and `unit: str`. Pydantic ensures the LLM provides valid values, and rejects invalid calls like `city: 123`.

---

### 🧪 5. Code Example (LangChain Tool Input Schema)

```python
from pydantic import BaseModel, Field
from langchain.tools import tool

# ✅ Define schema using Pydantic
class WeatherToolInput(BaseModel):
    city: str = Field(..., description="City name to get weather for")
    unit: str = Field(default="Celsius", description="Temperature unit")

# ✅ Attach schema to a tool
@tool(args_schema=WeatherToolInput)
def get_weather(city: str, unit: str = "Celsius") -> str:
    return f"The weather in {city} is 28° {unit}"

# ✅ Now GPT or LangGraph agent can use it with structured validation
```

---



---

# 📦 **2. BaseModel (Backbone of AI Tool Schemas)**

---

### ✅ 1. Definition

`BaseModel` is the **core class** in Pydantic v2 used to define **typed and validated data models**.
It auto-parses input types, applies constraints, and enables structured data for tools, agents, chains, and memory.

---

### 🧰 2. Built-in Functions & Configs

| Method / Config        | Description                                                         |
| ---------------------- | ------------------------------------------------------------------- |
| `BaseModel()`          | Create a new model with validated data                              |
| `.model_validate(obj)` | Parses and validates external input                                 |
| `.model_dump()`        | Converts the model to a Python dict                                 |
| `.model_dump_json()`   | Serializes the model as JSON string                                 |
| `.model_copy()`        | Returns a new copy (with optional overrides)                        |
| `ConfigDict`           | Allows customizing behavior (e.g., `frozen=True`, `extra='forbid'`) |
| `.model_config`        | Per-model configuration dictionary                                  |

---

### 🤖 3. When & Where Used (LangChain + LangGraph)

| Use Case Area        | Description                                              |
| -------------------- | -------------------------------------------------------- |
| ✅ LangChain Tools    | Used to define `args_schema` for @tool decorators        |
| ✅ Agent Tool Calling | Structures input/output for LLM tool usage               |
| ✅ LangGraph State    | State classes inherit from BaseModel                     |
| ✅ Agent Memory       | Defines slot structures with type safety                 |
| ✅ Output Parsers     | Used to auto-parse LLM responses into structured formats |

---

### 🌐 4. Real-Time Use Case

You’re building a **LangGraph support agent**. Its memory state includes:

* `user_email: str`
* `issue_type: str`
* `priority: Optional[str]`

By subclassing `BaseModel`, the state is validated every step in the graph, ensuring data consistency and agent robustness.

---

### 🧪 5. Full Code Implementation

#### ✅ Example 1: LangChain Tool Input Schema

```python
from pydantic import BaseModel, Field
from langchain.tools import tool

class MathInput(BaseModel):
    x: int = Field(..., description="First number")
    y: int = Field(..., description="Second number")

@tool(args_schema=MathInput)
def add(x: int, y: int) -> int:
    return x + y

# LangChain will validate inputs passed from LLM before running the tool
```

---

#### ✅ Example 2: LangGraph Agent State

```python
from pydantic import BaseModel
from langgraph.graph import StateGraph, END

# Define agent memory/state
class SupportState(BaseModel):
    user_email: str
    issue_type: str
    priority: str = "normal"

# Define a dummy node that handles state
def log_issue(state: SupportState) -> SupportState:
    print(f"Received issue from {state.user_email}")
    return state

# Build the graph
builder = StateGraph(SupportState)
builder.add_node("log", log_issue)
builder.set_entry_point("log")
builder.set_finish_point("log")

graph = builder.compile()
graph.invoke(SupportState(user_email="john@example.com", issue_type="login"))
```

---

### ✅ Bonus Config: Frozen State Example (Prevents Mutation)

```python
from pydantic import BaseModel, ConfigDict

class FrozenState(BaseModel):
    model_config = ConfigDict(frozen=True)
    name: str

state = FrozenState(name="agent")
# state.name = "updated"  # ❌ Raises error: cannot modify frozen model
```

---



---

# 📄 **3. Model Fields (Types, Constraints, and Metadata)**

---

### ✅ 1. Definition

**Model Fields** in Pydantic are **typed attributes** inside a `BaseModel` that define:

* the structure,
* constraints (like `min_length`, `ge`, etc.),
* defaults,
* and documentation metadata (like `description`, `examples`).

They are declared using:

```python
from pydantic import Field
```

---

### 🧰 2. Built-in Functions & Field Options

| Feature          | Example                                      | Description                           |
| ---------------- | -------------------------------------------- | ------------------------------------- |
| `Field(...)`     | `name: str = Field(...)`                     | Required field                        |
| `default=`       | `count: int = Field(0)`                      | Optional with default                 |
| `description=`   | `Field(..., description="Enter your email")` | Tool/prompt metadata                  |
| `ge=0`, `le=100` | `score: int = Field(..., ge=0, le=100)`      | Numeric constraints                   |
| `min_length=3`   | `name: str = Field(..., min_length=3)`       | String length constraint              |
| `alias=`         | `Field(..., alias="userId")`                 | Alternate name (e.g., for LLM output) |
| `examples=`      | `Field(..., examples=["foo"])`               | Used for UI or prompting context      |

👉 Works with: `str`, `int`, `float`, `bool`, `list`, `dict`, `Optional[]`, `Literal`, `Annotated`, etc.

---

### 🤖 3. When & Where Used (LangChain + LangGraph)

| Use Case                 | Description                                                        |
| ------------------------ | ------------------------------------------------------------------ |
| ✅ Tool Parameters        | Define expected inputs (type, required/optional, docs)             |
| ✅ LangGraph State Fields | Define memory/state variables and constraints                      |
| ✅ Output Parsers         | Describe the fields to expect from LLM output                      |
| ✅ Tool Metadata          | Enhance tool discoverability by LLM with `description`, `examples` |
| ✅ Prompt Templates       | Fields injected into prompts with type safety and doc hints        |

---

### 🌐 4. Real-Time Use Case

You're building a **Document Search tool** in LangChain. The tool accepts a query and number of results. You want to:

* Require `query: str`
* Limit `top_k: int` to values between 1 and 10

This can be safely defined using `Field(..., ge=1, le=10)`, so the agent doesn't hallucinate an invalid value like -5 or 100.

---

### 🧪 5. Full Code Implementation

#### ✅ Example 1: LangChain Tool Input Fields with Constraints

```python
from pydantic import BaseModel, Field
from langchain.tools import tool

class SearchInput(BaseModel):
    query: str = Field(..., description="Search query string")
    top_k: int = Field(5, ge=1, le=10, description="Number of top results (1–10)")

@tool(args_schema=SearchInput)
def search(query: str, top_k: int) -> str:
    return f"Searching '{query}' and returning top {top_k} results."

# LLM will be forced to use only valid values
```

---

#### ✅ Example 2: LangGraph State with Field Metadata

```python
from pydantic import BaseModel, Field

class TicketState(BaseModel):
    email: str = Field(..., description="Customer email ID")
    issue: str = Field(..., min_length=10, description="Detailed issue description")
    priority: str = Field(default="normal", description="Priority level")

# LangGraph will enforce these field rules at every step
```

---

#### ✅ Example 3: Field Aliasing for Prompt Format

```python
class InputModel(BaseModel):
    user_id: str = Field(..., alias="userId", description="Unique ID of the user")

# If LLM response returns {"userId": "abc123"}, it maps to `user_id` in the model
parsed = InputModel.model_validate({"userId": "abc123"})
print(parsed.user_id)  # ✅ abc123
```

---

### ✅ Summary

| What Fields Help You Do | How It Helps in GenAI                 |
| ----------------------- | ------------------------------------- |
| Constrain inputs        | Prevent LLM errors/hallucination      |
| Add metadata            | Better prompt injection and UI        |
| Enforce types           | Safe execution in LangGraph/LangChain |
| Default/Optional logic  | Build flexible agents                 |

---



---

# 🎯 **4. Validation in Pydantic v2**

---

### ✅ 1. Definition

**Validation** in Pydantic is the process of checking whether the data conforms to type constraints, field rules, and custom logic.
Pydantic v2 allows:

* Built-in validation (via `Field`)
* Advanced validation (via decorators like `@field_validator` and `@model_validator`)

This ensures agents/tools **never operate on invalid inputs**, and **LLMs are held accountable**.

---

### 🧰 2. Built-in Functions & Validators

#### 🔹 Field-Level Validators

```python
@field_validator("field_name", mode="before" or "after")
def validate_field(cls, value): ...
```

| Parameter                | Description                   |
| ------------------------ | ----------------------------- |
| `field_name`             | Name of the field to validate |
| `mode="before"`          | Runs before type coercion     |
| `mode="after"` (default) | Runs after coercion           |

---

#### 🔹 Model-Level Validators

```python
@model_validator(mode="before" or "after")
def validate_model(cls, values): ...
```

\| Use case | Combine or cross-validate multiple fields |

---

#### 🔹 Built-in Field Constraints (via `Field`)

* `min_length`, `max_length`, `ge`, `le`, `regex`
* `literal`, `enum`, `optional`

---

### 🤖 3. When & Where Used (LangChain + LangGraph)

| Use Case                | Description                                       |
| ----------------------- | ------------------------------------------------- |
| ✅ Tool Input Validation | Prevent invalid agent inputs (e.g., empty search) |
| ✅ Cross-Field Logic     | e.g., `if x == "A" then y must be True`           |
| ✅ Memory Checks         | Ensure state variables are consistent             |
| ✅ Tool Trigger Safety   | Only allow tool execution if inputs are correct   |
| ✅ Prompt Guards         | Validate prompt response before use               |

---

### 🌐 4. Real-Time Use Case

You have a tool in LangChain to **generate meeting links**.

* If `platform == "Zoom"` → `email` is required.
  You use a **model validator** to enforce this logic, preventing GPT-4 from skipping required values when calling tools.

---

### 🧪 5. Full Code Implementation

#### ✅ Example 1: Field Validator (after LLM input)

```python
from pydantic import BaseModel, Field, field_validator
from langchain.tools import tool

class EmailInput(BaseModel):
    email: str = Field(..., description="User email")

    @field_validator("email")
    def check_email_format(cls, v):
        if "@" not in v:
            raise ValueError("Invalid email format")
        return v

@tool(args_schema=EmailInput)
def send_invite(email: str):
    return f"Invite sent to {email}"
```

🧠 GPT can’t call this tool without a valid `email@domain.com`.

---

#### ✅ Example 2: Model Validator (Cross-field)

```python
from pydantic import BaseModel, Field, model_validator

class MeetingInput(BaseModel):
    platform: str = Field(..., description="Platform (Zoom or Teams)")
    email: str | None = Field(None, description="Required if Zoom")

    @model_validator(mode="after")
    def validate_zoom_email(cls, values):
        if values.platform == "Zoom" and not values.email:
            raise ValueError("Email is required for Zoom meetings")
        return values
```

🛡️ Used inside LangGraph step or LangChain tool, this prevents LLM from triggering invalid logic.

---

#### ✅ Example 3: Before-Mode Validator (Raw Input Cleaning)

```python
class UserInput(BaseModel):
    name: str

    @field_validator("name", mode="before")
    def strip_whitespace(cls, v):
        return v.strip()
```

🧼 Useful for cleaning prompt responses from LLMs.

---

### ✅ Summary Table

| Type                    | Purpose                | Use in GenAI                    |
| ----------------------- | ---------------------- | ------------------------------- |
| `@field_validator`      | Field-level checks     | Validate individual tool params |
| `@model_validator`      | Cross-field validation | Validate full tool input state  |
| Constraints via `Field` | Quick validation       | Auto-limit LLM/tool usage       |

---



---

# 🧬 **5. Model Composition in Pydantic v2**

---

### ✅ 1. Definition

**Model Composition** in Pydantic refers to combining models together using:

* **Inheritance** (reusability),
* **Nested models** (structured inputs/outputs),
* **Recursive models** (self-referencing),
* and **Generic models** (optional, advanced pattern).

These techniques are essential when you need to build **multi-field states** or **tool schemas that contain other schemas** — like nested dictionaries with validation.

---

### 🧰 2. Built-in Features

| Feature             | Syntax                    | Description                                              |
| ------------------- | ------------------------- | -------------------------------------------------------- |
| 🔁 Inheritance      | `class B(A)`              | Reuse base schemas                                       |
| 📦 Nested models    | Field with model type     | Validate deeply structured inputs                        |
| 🔁 Recursive models | `Optional[List['Model']]` | Self-referencing chains (requires update\_forward\_refs) |
| 🔢 Generics         | `GenericModel[T]`         | Template-like data models                                |

---

### 🤖 3. When & Where Used (LangChain + LangGraph)

| Use Case           | Description                                                          |
| ------------------ | -------------------------------------------------------------------- |
| ✅ Tool Composition | A tool input model contains another model                            |
| ✅ Agent Memory     | Store memory as nested object (e.g., user profile, chat history)     |
| ✅ LangGraph State  | Model state with deeply structured properties                        |
| ✅ Planner Outputs  | Reasoning chains return a list of nested task models                 |
| ✅ Output Parsing   | LLM output parsed into multiple nested sections (metadata + content) |

---

### 🌐 4. Real-Time Use Case

You’re building a **LangGraph Planner Agent**.

* The planner returns a list of steps.
* Each step contains a tool name and its arguments, which may themselves be structured.

Using **nested models and recursive composition**, you can model this plan in a clean, validated form.

---

### 🧪 5. Full Code Implementation

#### ✅ Example 1: Nested Model (Tool Schema with sub-model)

```python
from pydantic import BaseModel, Field

class Coordinates(BaseModel):
    latitude: float = Field(..., ge=-90, le=90)
    longitude: float = Field(..., ge=-180, le=180)

class LocationRequest(BaseModel):
    city: str
    coordinates: Coordinates

# ✅ Tool schema or LangGraph state can now use LocationRequest directly
data = {
    "city": "San Francisco",
    "coordinates": {"latitude": 37.77, "longitude": -122.42}
}

parsed = LocationRequest.model_validate(data)
print(parsed.coordinates.latitude)  # 37.77
```

---

#### ✅ Example 2: Inheritance for Shared Fields

```python
class BaseToolInput(BaseModel):
    user_id: str
    session_id: str

class SearchToolInput(BaseToolInput):
    query: str
    top_k: int = 5
```

🧠 Useful for LangChain tools with common inputs (user/session tracking).

---

#### ✅ Example 3: Recursive Model (for Planning or Graphs)

```python
from __future__ import annotations  # Required for self-reference
from typing import List, Optional
from pydantic import BaseModel

class Task(BaseModel):
    name: str
    subtasks: Optional[List[Task]] = None

# Needed to resolve forward references
Task.model_rebuild()

# Example input
plan = {
    "name": "Build App",
    "subtasks": [
        {"name": "Design UI"},
        {"name": "Write Backend", "subtasks": [
            {"name": "Setup DB"},
            {"name": "Create API"}
        ]}
    ]
}

parsed = Task.model_validate(plan)
print(parsed.subtasks[1].subtasks[0].name)  # Setup DB
```

---

#### ✅ Example 4: LangGraph State with Nested Memory

```python
class UserMemory(BaseModel):
    email: str
    preferences: dict

class AgentState(BaseModel):
    user: UserMemory
    current_task: str
```

🔁 Can be serialized, validated, and passed across nodes in LangGraph.

---

### ✅ Summary

| Composition Type         | Use in GenAI                        |
| ------------------------ | ----------------------------------- |
| Nested Models            | Tool inputs with sub-fields         |
| Inheritance              | Shared tool inputs or agent context |
| Recursive Models         | Planning trees or toolchains        |
| Composition + Validation | Structuring LangGraph state trees   |

---



---

# 📤 **6. Serialization & Deserialization in Pydantic v2**

---

### ✅ 1. Definition

**Serialization** is the process of converting a Pydantic model to a `dict` or `JSON` string so it can be passed between agents, saved to memory, or logged.

**Deserialization** is converting external data (LLM output, tool input, memory state) into a validated model.

In Pydantic v2, this is done through:

* `.model_dump()`
* `.model_dump_json()`
* `.model_validate()`

---

### 🧰 2. Built-in Functions & Methods

| Function                   | Purpose                                      |
| -------------------------- | -------------------------------------------- |
| `model_dump()`             | Serialize model → Python `dict`              |
| `model_dump_json()`        | Serialize model → `JSON` string              |
| `model_validate(data)`     | Parse dict/JSON → Model with validation      |
| `model_copy(update={...})` | Clone model with optional updates            |
| `@field_serializer`        | Customize how specific fields are serialized |

---

### 🤖 3. When & Where Used (LangChain + LangGraph)

| Area                     | Purpose                                       |
| ------------------------ | --------------------------------------------- |
| ✅ LangGraph State Memory | Persist and restore agent state between steps |
| ✅ LLM Tool Calls         | Convert tool output to JSON or dict           |
| ✅ Input Parsing          | Convert LLM raw output → structured model     |
| ✅ Debugging              | Log and visualize the model                   |
| ✅ Prompt Templates       | Inject data into structured prompts           |
| ✅ OutputParser           | Use model to validate and parse LLM outputs   |

---

### 🌐 4. Real-Time Use Case

A LangGraph agent must pass state between nodes in JSON format. You need to:

* Dump the current memory state before saving it.
* Validate the restored state before reusing it.

This guarantees the state is never corrupted or malformed during execution.

---

### 🧪 5. Full Code Implementation

---

#### ✅ Example 1: Serializing a LangGraph Agent State

```python
from pydantic import BaseModel, Field

class AgentState(BaseModel):
    user_id: str
    current_step: str
    context: dict = Field(default_factory=dict)

# Create instance
state = AgentState(user_id="U123", current_step="fetch")

# Serialize to dict (e.g., for memory save or prompt inject)
as_dict = state.model_dump()
print(as_dict)

# Serialize to JSON (e.g., send to external service or frontend)
as_json = state.model_dump_json()
print(as_json)
```

---

#### ✅ Example 2: Deserialization from Input (LLM Output)

```python
data = {
    "user_id": "U123",
    "current_step": "fetch",
    "context": {"intent": "weather"}
}

validated_state = AgentState.model_validate(data)
print(validated_state.context["intent"])  # ✅ weather
```

---

#### ✅ Example 3: Custom Serialization of a Field

```python
from pydantic import BaseModel, Field, field_serializer
from datetime import datetime

class TimeStampedOutput(BaseModel):
    response: str
    timestamp: datetime = Field(default_factory=datetime.utcnow)

    @field_serializer("timestamp")
    def format_time(cls, v):
        return v.isoformat()

output = TimeStampedOutput(response="Done")
print(output.model_dump())  # Timestamp will be in ISO format
```

---

#### ✅ Example 4: LangChain OutputParser Using Pydantic Model

```python
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel

class ResponseModel(BaseModel):
    answer: str
    confidence: float

parser = PydanticOutputParser(pydantic_object=ResponseModel)

# Example LLM response
raw = '{"answer": "Yes", "confidence": 0.91}'
parsed = parser.parse(raw)
print(parsed.answer)  # ✅ "Yes"
```

---

### ✅ Summary Table

| Method              | Usage in GenAI                                        |
| ------------------- | ----------------------------------------------------- |
| `model_dump()`      | Save LangGraph state / inject into prompt             |
| `model_dump_json()` | Persist to Redis or remote DB                         |
| `model_validate()`  | Ingest tool input or LLM output safely                |
| `@field_serializer` | Customize serialization of timestamps, decimals, etc. |

---


---

# 🛡️ **7. Strict Mode & Type Safety in Pydantic v2**

---

### ✅ 1. Definition

**Strict Mode and Type Safety** in Pydantic v2 enforce that **inputs match exactly the expected types** — no coercion allowed.

🔒 For example:

* `"5"` won’t be accepted as an `int`
* `None` won’t be accepted unless declared as `Optional[...]`

This is **vital in Agentic AI** to prevent LLMs from “hallucinating” valid-looking, but **structurally invalid** tool inputs or state.

---

### 🧰 2. Built-in Tools & Configurations

| Feature                                      | Description                                                     |
| -------------------------------------------- | --------------------------------------------------------------- |
| `StrictStr`, `StrictInt`, `StrictBool`, etc. | Accept only exact types                                         |
| `ConfigDict(strict=True)`                    | Enforce strict mode globally on the model                       |
| `extra='forbid'`                             | Disallow extra fields                                           |
| `frozen=True`                                | Make models immutable                                           |
| `allow_inf_nan=False`                        | Disallow `NaN` or `Infinity`                                    |
| `model_config`                               | Local model-level configuration (instead of class Config in v1) |

---

### 🤖 3. When & Where Used (LangChain + LangGraph)

| Use Case                      | Description                                                   |
| ----------------------------- | ------------------------------------------------------------- |
| ✅ Tool Input Validation       | Block malformed inputs like `"5"` for an `int`                |
| ✅ LangGraph State Consistency | Prevent LLM or user logic from injecting extra or bad data    |
| ✅ Agent Tool-Safety           | Harden tools against bad LLM behavior                         |
| ✅ Critical Steps              | Use strict types for financial, scheduling, or security tasks |
| ✅ Memory Mutability           | Use `frozen=True` to prevent unwanted state changes mid-run   |

---

### 🌐 4. Real-Time Use Case

Your LLM planner agent returns `"10"` as a string for a `page_count: int` tool input.

Without strict mode, this may be coerced and silently accepted.
With strict mode, it will raise a validation error — forcing your system to either correct or reject the invalid LLM behavior.

---

### 🧪 5. Full Code Implementation

---

#### ✅ Example 1: Enforcing Strict Types

```python
from pydantic import BaseModel, StrictInt, StrictStr

class ToolInput(BaseModel):
    query: StrictStr
    count: StrictInt  # 👈 Must be int, not string

# Valid input
ToolInput.model_validate({"query": "LLM safety", "count": 5})  # ✅

# Invalid input (LLM returns string)
ToolInput.model_validate({"query": "test", "count": "5"})  
# ❌ ValidationError: value is not a valid integer
```

---

#### ✅ Example 2: Strict Model Config

```python
from pydantic import BaseModel, ConfigDict

class SecureState(BaseModel):
    model_config = ConfigDict(strict=True, extra='forbid')  # 👈 Enforce strict mode globally
    step: int
    user: str

# Invalid extra field
SecureState.model_validate({"step": 1, "user": "Alice", "bad_field": "oops"})
# ❌ Error: Extra fields not permitted
```

---

#### ✅ Example 3: Frozen Models for LangGraph State

```python
from pydantic import BaseModel, ConfigDict

class FrozenAgentState(BaseModel):
    model_config = ConfigDict(frozen=True)
    user_id: str
    task: str

state = FrozenAgentState(user_id="U123", task="fetch")
# state.task = "update"  # ❌ TypeError: Cannot assign to frozen model
```

🔐 Ensures the agent’s state is **immutable** during execution.

---

### ✅ Summary Table

| Feature                  | Why It Matters in GenAI                |
| ------------------------ | -------------------------------------- |
| `StrictStr`, `StrictInt` | Blocks unsafe coercion from LLM inputs |
| `strict=True`            | Ensures no hidden errors slip in       |
| `extra='forbid'`         | Prevents hallucinated inputs           |
| `frozen=True`            | Makes memory/state read-only           |
| `allow_inf_nan=False`    | Ensures mathematical safety            |

---




---

# 🔗 **8. Advanced Features in Pydantic v2**

---

### ✅ 1. Definition

Pydantic v2 offers several advanced features that allow you to:

* Dynamically switch between model types (Union/Discriminated Unions),
* Store internal state or metadata (`PrivateAttr`),
* Define read-only computed fields (`@computed_field`),
* Introspect fields for dynamic tool wiring or graph routing.

These are useful in **multi-agent routing**, **memory-controlled LangGraph edges**, and **adaptive agent state machines**.

---

### 🧰 2. Built-in Features & Functions

| Feature                  | Purpose                                  | Syntax                                                   |
| ------------------------ | ---------------------------------------- | -------------------------------------------------------- |
| **Discriminated Unions** | Switch between model types using a field | `Literal[...]`, `discriminator="type"`                   |
| **Private Attributes**   | Store internal, non-serializable values  | `PrivateAttr()`                                          |
| **Computed Fields**      | Create read-only dynamic fields          | `@computed_field(return_type=...)`                       |
| **Introspection**        | Access model internals                   | `__pydantic_fields__`, `__annotations__`, `model_fields` |

---

### 🤖 3. When & Where Used (LangChain + LangGraph)

| Use Case                  | Description                                                          |
| ------------------------- | -------------------------------------------------------------------- |
| ✅ Multi-Agent Routing     | Use Discriminated Union to define which agent to route to            |
| ✅ LangGraph Memory        | Use PrivateAttr to track token count or LLM call logs                |
| ✅ Read-only Metadata      | Computed fields like `created_at`, `step_count`, `token_budget_used` |
| ✅ Conditional State Logic | Use model introspection to choose graph edge based on model content  |
| ✅ Agent Type Handling     | Model different agent inputs/outputs using Union types               |

---

### 🌐 4. Real-Time Use Case

In LangGraph, your memory state needs to support **two agent types**: `PlannerAgent` and `ToolAgent`.

Using a **discriminated union**, LangGraph can route to the right node depending on the agent type field in the state.
You also want to track the **token usage internally** (not serialized) using a `PrivateAttr`.

---

### 🧪 5. Full Code Implementation

---

#### ✅ Example 1: Discriminated Union (Multi-Agent Routing)

```python
from pydantic import BaseModel, Field
from typing import Literal, Union

class PlannerAgent(BaseModel):
    type: Literal["planner"]
    plan: str

class ToolAgent(BaseModel):
    type: Literal["tool"]
    tool_name: str
    args: dict

AgentState = Union[PlannerAgent, ToolAgent]  # 👈 Switches based on `type`

# Validate incoming state
raw = {"type": "tool", "tool_name": "search", "args": {"query": "weather"}}
agent = ToolAgent.model_validate(raw)
print(agent.tool_name)  # ✅ "search"
```

📌 In LangGraph, use this union to determine node logic dynamically.

---

#### ✅ Example 2: Private Attribute (Non-Serialized State)

```python
from pydantic import BaseModel, PrivateAttr

class MemoryState(BaseModel):
    user_id: str
    _token_count: int = PrivateAttr(default=0)

    def update_token_usage(self, tokens: int):
        self._token_count += tokens

state = MemoryState(user_id="U456")
state.update_token_usage(100)
print(state._token_count)  # ✅ 100

print(state.model_dump())  # ✅ No "_token_count" — it's private
```

🔒 Safe to use for tracking behind-the-scenes logic or metadata.

---

#### ✅ Example 3: Computed Field (Read-only Metadata)

```python
from pydantic import BaseModel, computed_field
from datetime import datetime

class UserState(BaseModel):
    name: str

    @computed_field(return_type=str)
    def created_at(self) -> str:
        return datetime.utcnow().isoformat()

user = UserState(name="John")
print(user.created_at)  # ✅ Automatically generated timestamp
```

🔄 You can use this in LangGraph states to add runtime-only data like time, tokens, etc.

---

#### ✅ Example 4: Model Introspection (For Dynamic Tool Creation)

```python
class ToolInput(BaseModel):
    query: str
    top_k: int = 5

print(ToolInput.__pydantic_fields__.keys())  # dict_keys(['query', 'top_k'])

# Useful for LangGraph dynamic node wiring
```

---

### ✅ Summary Table

| Feature                | Benefit in GenAI / LangGraph           |
| ---------------------- | -------------------------------------- |
| `Discriminated Unions` | Dynamically switch agent/tool types    |
| `PrivateAttr`          | Store runtime-only info (tokens, time) |
| `@computed_field`      | Add derived read-only fields           |
| `__pydantic_fields__`  | Enable graph and tool introspection    |

---



---

# 📚 **9. Error Handling in Pydantic v2**

---

### ✅ 1. Definition

**Error Handling** in Pydantic ensures that any **invalid, malformed, or unexpected input** gets caught with clear, structured error messages.
Pydantic raises a `ValidationError` whenever data fails to meet the model’s rules — and you can catch, inspect, or log it.

This is essential for:

* 🛡️ Preventing LangChain tools from silently failing
* 🔁 Implementing retries in LangGraph
* 📬 Giving LLMs feedback when their output doesn't validate

---

### 🧰 2. Key Error Classes & Methods

| Class / Method    | Description                                                     |
| ----------------- | --------------------------------------------------------------- |
| `ValidationError` | Raised on invalid `.model_validate()` calls                     |
| `.errors()`       | Returns list of field-level error dicts                         |
| `.error_count()`  | Returns number of errors                                        |
| `.json()`         | Serializes the error as JSON (great for debugging/LLM feedback) |
| `try/except`      | Catch validation issues in tool execution or LangGraph steps    |

---

### 🤖 3. When & Where Used (LangChain + LangGraph)

| Use Case                | Description                                            |
| ----------------------- | ------------------------------------------------------ |
| ✅ Tool Call Input Guard | Catch bad LLM input before tool executes               |
| ✅ LangGraph Node Safety | Prevent bad state updates or transitions               |
| ✅ Retry Logic           | Detect failure and re-ask LLM for corrected output     |
| ✅ Logging + Debugging   | Log error tracebacks for observability                 |
| ✅ Feedback Loop         | Return schema error message to LLM for self-correction |

---

### 🌐 4. Real-Time Use Case

You’re using a **LangChain Tool** that expects:

```python
{"email": "user@example.com", "age": 30}
```

But GPT mistakenly outputs:

```python
{"email": 1234, "age": "old"}
```

Instead of crashing, you use Pydantic’s validation to:

* Catch the error
* Print a user-friendly message
* Retry the call or correct the LLM

---

### 🧪 5. Full Code Implementation

---

#### ✅ Example 1: Basic Validation Error Catching

```python
from pydantic import BaseModel, ValidationError

class UserInput(BaseModel):
    email: str
    age: int

bad_input = {"email": 1234, "age": "old"}

try:
    validated = UserInput.model_validate(bad_input)
except ValidationError as e:
    print("⚠️ Validation failed!")
    print(e.errors())  # List of errors
    print(e.json())    # JSON string (useful for LLM feedback)
```

🧠 This allows you to **catch malformed tool inputs** before execution.

---

#### ✅ Example 2: Use Inside LangChain Tool

```python
from langchain.tools import tool
from pydantic import BaseModel, ValidationError, Field

class WeatherInput(BaseModel):
    city: str = Field(..., description="City to search")
    unit: str = Field(default="Celsius")

@tool(args_schema=WeatherInput)
def get_weather(city: str, unit: str = "Celsius") -> str:
    return f"Weather in {city} is 25° {unit}"

# Simulate tool call from LLM
raw_input = {"city": 999}  # invalid input

try:
    validated = WeatherInput.model_validate(raw_input)
    print(get_weather(**validated.model_dump()))
except ValidationError as e:
    print("🛑 GPT provided invalid tool input:")
    print(e.errors())
```

---

#### ✅ Example 3: LangGraph Retry Logic with Validation

```python
from langgraph.graph import StateGraph, END
from pydantic import BaseModel, ValidationError

class AgentState(BaseModel):
    query: str

def validate_and_continue(state: dict):
    try:
        valid = AgentState.model_validate(state)
        return valid
    except ValidationError as e:
        print("🧠 Retry needed due to invalid state:", e.errors())
        return {"query": "DEFAULT"}  # Or retry node logic

graph = StateGraph(AgentState)
graph.add_node("check", validate_and_continue)
graph.set_entry_point("check")
graph.set_finish_point("check")
graph = graph.compile()

graph.invoke({"query": 123})  # Invalid query triggers fallback logic
```

---

### ✅ Summary Table

| Feature           | Benefit in GenAI                           |
| ----------------- | ------------------------------------------ |
| `ValidationError` | Catch invalid LLM input or state           |
| `.errors()`       | Inspect what went wrong field-by-field     |
| `.json()`         | Use error in LLM feedback prompts          |
| Retry logic       | Ask LLM to regenerate with corrected input |
| Logging           | Debug tool failures in LangGraph pipelines |

---



---

# 🔁 **10. Integration with LangChain and LangGraph**

---

### ✅ 1. Definition

This section explains **how Pydantic v2 is deeply integrated** with:

* 🔧 LangChain tools
* 🧠 LangGraph state machines
* 🪄 Output parsers and memory handlers
* 🧩 LLM function calling & schema validation

Pydantic acts as the **schema backbone** for input validation, memory consistency, and safe execution paths in AI agents.

---

### 🧰 2. Pydantic Roles in LangChain & LangGraph

| Layer                    | Usage                                                        |
| ------------------------ | ------------------------------------------------------------ |
| 🛠️ Tools                | `@tool(args_schema=YourModel)`                               |
| 🗣️ LLM Function Calling | Auto-generated OpenAI function schemas                       |
| 🧠 State Management      | LangGraph agent state = `BaseModel` subclass                 |
| 📦 Output Parsing        | Use `PydanticOutputParser` to parse structured LLM responses |
| 🪪 Memory                | Define structured memory with nested/composed models         |
| 🔁 Retry & Feedback      | Catch `ValidationError` and trigger retry logic              |

---

### 🤖 3. Where It’s Used (In Practice)

| Feature             | LangChain    | LangGraph                                  |
| ------------------- | ------------ | ------------------------------------------ |
| Tool input schema   | ✅            | ✅                                          |
| Tool return schema  | ✅ (custom)   | ✅                                          |
| Agent memory/state  | ❌            | ✅ (core requirement)                       |
| Output parser       | ✅            | ✅                                          |
| Retry mechanism     | Limited      | ✅ (node logic or error flow)               |
| Multi-agent planner | Experimental | ✅ (with discriminated unions + validators) |

---

### 🌐 4. Real-Time Use Case

You're building a **LangGraph-based multi-agent system**:

1. The `Planner` agent decides the next step.
2. The plan is validated with a Pydantic model.
3. If the plan is invalid, LangGraph retries.
4. A downstream `ToolExecutor` uses `@tool(args_schema=YourInputModel)` to execute tools.
5. The tool result is validated again and stored in structured memory (BaseModel).

This entire pipeline uses **only Pydantic** models for inputs, outputs, memory, retry, and edge routing.

---

### 🧪 5. Full Code Integration Example

---

#### ✅ Step 1: Define Tool Input Schema

```python
from pydantic import BaseModel, Field
from langchain.tools import tool

class SearchInput(BaseModel):
    query: str = Field(..., description="Search query")
    top_k: int = Field(default=5, ge=1, le=10)

@tool(args_schema=SearchInput)
def search_tool(query: str, top_k: int = 5) -> str:
    return f"Searching for {query} (top {top_k} results)"
```

---

#### ✅ Step 2: LangGraph State with Pydantic

```python
from langgraph.graph import StateGraph
from pydantic import BaseModel

class AgentState(BaseModel):
    step: str
    memory: dict
```

---

#### ✅ Step 3: Node with Pydantic Validation & Retry

```python
from pydantic import ValidationError

def process_step(state: AgentState) -> AgentState:
    try:
        # Ensure memory contains valid keys
        if "query" not in state.memory:
            raise ValidationError("Missing query in memory")
        return state
    except ValidationError as e:
        print("❌ Invalid memory structure:", e)
        # Fix it or send to fallback edge
        return AgentState(step="fallback", memory={"query": "default"})
```

---

#### ✅ Step 4: Graph Wiring

```python
builder = StateGraph(AgentState)
builder.add_node("process", process_step)
builder.set_entry_point("process")
builder.set_finish_point("process")
graph = builder.compile()

graph.invoke(AgentState(step="start", memory={}))  # Triggers retry logic
```

---

#### ✅ Step 5: Output Parser with Pydantic

```python
from langchain.output_parsers import PydanticOutputParser

class AnswerModel(BaseModel):
    answer: str
    confidence: float

parser = PydanticOutputParser(pydantic_object=AnswerModel)

llm_response = '{"answer": "42", "confidence": 0.99}'
parsed = parser.parse(llm_response)
print(parsed.answer)  # ✅ 42
```

---

### ✅ Summary Diagram

```text
+-------------------+     Pydantic Model      +----------------------+
|   LangChain Tool  | <--------------------> |     Input Schema     |
+-------------------+                        +----------------------+
        |
        v
+-------------------+                        +----------------------+
|  LLM Function Call| <--------------------> |  JSON Function Spec  |
+-------------------+                        +----------------------+
        |
        v
+-------------------+                        +----------------------+
| LangGraph Node    | <--------------------> |   AgentState (Model) |
+-------------------+                        +----------------------+
        |
        v
+-------------------+                        +----------------------+
|  Output Parser    | <--------------------> |    Output Model      |
+-------------------+                        +----------------------+
```

---




---

## 🧪 **11. Testing & Debugging in Agentic AI (with Pydantic)**

We’ll cover:

1. ✅ How to **simulate tool inputs** for testing
2. ✅ How to **unit test schemas** in isolation
3. ✅ How to **inspect & debug state** using `.model_dump()`
4. ✅ Best practices for **multi-agent test coverage**

